In [1]:
! pip3 install rdkit

import numpy as np
import rdkit
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 36.5 MB/s eta 0:00:00


In [3]:
from google.colab import files
data=files.upload()

Saving drugs_chembl.csv to drugs_chembl.csv


In [5]:
df = pd.read_csv('drugs_chembl.csv',sep=';',on_bad_lines='skip')
df.head()

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,Action Type,Standard Text Value
0,CHEMBL253309,NaN,NaN,563.79,1.0,3.30,"3, cyclo(-L-Am7(S2Py)-Aib-L-Leu-D-Pro-)",CC(C)C[C@@H]1NC(=O)C(C)(C)NC(=O)[C@H](CCCCCSSc...,IC50,'=',...,SINGLE PROTEIN,CHEMBL1140755,1,Scientific Literature,Bioorg Med Chem,2008,NaN,NaN,NaN,NaN
1,CHEMBL269935,NaN,NaN,289.33,0.0,4.31,2,O=C(Nc1cc(-c2ccccc2)ccc1O)c1ccccc1,IC50,'>',...,SINGLE PROTEIN,CHEMBL1143185,1,Scientific Literature,Bioorg Med Chem Lett,2008,NaN,NaN,NaN,NaN
2,CHEMBL437029,NaN,NaN,521.62,2.0,5.43,56,C[C@H]1CN(c2ccc(C(=O)Nc3cc(-c4ccccc4)ccc3N)cn2...,IC50,'>',...,SINGLE PROTEIN,CHEMBL1143185,1,Scientific Literature,Bioorg Med Chem Lett,2008,NaN,NaN,NaN,NaN
3,CHEMBL3639926,NaN,NaN,411.55,0.0,4.88,BDBM110013,Cc1cc(C)c(Nc2ccc(C(=O)N(C)CCCCCCC(=O)NO)cc2)c(...,IC50,'=',...,SINGLE PROTEIN,CHEMBL3639257,37,BindingDB Database,NaN,2013,NaN,NaN,NaN,NaN
4,CHEMBL3693779,NaN,NaN,496.55,1.0,5.09,BDBM110029,COC(c1ccc(F)cc1)(c1ccc(F)cc1)c1ccc(C(=O)NCCCCC...,IC50,'=',...,SINGLE PROTEIN,CHEMBL3639257,37,BindingDB Database,NaN,2013,NaN,NaN,NaN,NaN


In [51]:
print(df.columns)

Index(['Molecule ChEMBL ID', 'Molecule Name', 'Molecule Max Phase',
       'Molecular Weight', '#RO5 Violations', 'AlogP', 'Compound Key',
       'Smiles', 'Standard Type', 'Standard Relation', 'Standard Value',
       'Standard Units', 'pChEMBL Value', 'Data Validity Comment', 'Comment',
       'Uo Units', 'Ligand Efficiency BEI', 'Ligand Efficiency LE',
       'Ligand Efficiency LLE', 'Ligand Efficiency SEI', 'Potential Duplicate',
       'Assay ChEMBL ID', 'Assay Description', 'Assay Type', 'BAO Format ID',
       'BAO Label', 'Assay Organism', 'Assay Tissue ChEMBL ID',
       'Assay Tissue Name', 'Assay Cell Type', 'Assay Subcellular Fraction',
       'Assay Parameters', 'Assay Variant Accession', 'Assay Variant Mutation',
       'Target ChEMBL ID', 'Target Name', 'Target Organism', 'Target Type',
       'Document ChEMBL ID', 'Source ID', 'Source Description',
       'Document Journal', 'Document Year', 'Cell ChEMBL ID', 'Properties',
       'Action Type', 'Standard Text Value'],
 

In [56]:
new_df = df[['Smiles','Standard Value']]

In [57]:
new_df

,Smiles,Standard Value
0,CC(C)C[C@@H]1NC(=O)C(C)(C)NC(=O)[C@H](CCCCCSSc...,230.0
1,O=C(Nc1cc(-c2ccccc2)ccc1O)c1ccccc1,50000.0
2,C[C@H]1CN(c2ccc(C(=O)Nc3cc(-c4ccccc4)ccc3N)cn2...,50000.0
3,Cc1cc(C)c(Nc2ccc(C(=O)N(C)CCCCCCC(=O)NO)cc2)c(...,35.0
4,COC(c1ccc(F)cc1)(c1ccc(F)cc1)c1ccc(C(=O)NCCCCC...,13.0
...,...,...
6483,O=C(CCCCCCC(=O)Nc1ccccc1)NO,33.0
6484,O=C(CCCCCCNC(=O)c1cnc(N(c2ccccc2)c2ccccc2)nc1)NO,4.7
6485,CNc1nc(Cl)nc2c1ncn2Cc1ccc(C(=O)Nc2ccccc2N)cc1,21800.0
6486,Cc1cc(C)c(CNC(=O)c2cc(-c3ccc(NC(=O)CCCCCCC(=O)...,180.0


In [68]:
new_df['pIC50'] = -np.log10(new_df['Standard Value'] / 1e9)  # Convert to µM if in nM

# Create a new DataFrame with only Smiles and pIC50
result_df = new_df[['Smiles', 'pIC50']]

# Display the result
print(result_df)
result_df.head()

                                                 Smiles     pIC50
0     CC(C)C[C@@H]1NC(=O)C(C)(C)NC(=O)[C@H](CCCCCSSc...  6.638272
1                    O=C(Nc1cc(-c2ccccc2)ccc1O)c1ccccc1  4.301030
2     C[C@H]1CN(c2ccc(C(=O)Nc3cc(-c4ccccc4)ccc3N)cn2...  4.301030
3     Cc1cc(C)c(Nc2ccc(C(=O)N(C)CCCCCCC(=O)NO)cc2)c(...  7.455932
4     COC(c1ccc(F)cc1)(c1ccc(F)cc1)c1ccc(C(=O)NCCCCC...  7.886057
...                                                 ...       ...
6483                        O=C(CCCCCCC(=O)Nc1ccccc1)NO  7.481486
6484   O=C(CCCCCCNC(=O)c1cnc(N(c2ccccc2)c2ccccc2)nc1)NO  8.327902
6485      CNc1nc(Cl)nc2c1ncn2Cc1ccc(C(=O)Nc2ccccc2N)cc1  4.661544
6486  Cc1cc(C)c(CNC(=O)c2cc(-c3ccc(NC(=O)CCCCCCC(=O)...  6.744727
6487  COc1cc(C(=O)c2csc(-c3ccc(C#CCCC(=O)Nc4ccccc4N)...  4.694004

[6488 rows x 2 columns]


<ipython-input-68-3f200c5af49f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['pIC50'] = -np.log10(new_df['Standard Value'] / 1e9)  # Convert to µM if in nM


,Smiles,pIC50
0,CC(C)C[C@@H]1NC(=O)C(C)(C)NC(=O)[C@H](CCCCCSSc...,6.638272
1,O=C(Nc1cc(-c2ccccc2)ccc1O)c1ccccc1,4.301030
2,C[C@H]1CN(c2ccc(C(=O)Nc3cc(-c4ccccc4)ccc3N)cn2...,4.301030
3,Cc1cc(C)c(Nc2ccc(C(=O)N(C)CCCCCCC(=O)NO)cc2)c(...,7.455932
4,COC(c1ccc(F)cc1)(c1ccc(F)cc1)c1ccc(C(=O)NCCCCC...,7.886057


In [69]:
result_df['pIC50'].max()

11.698970004336019

In [70]:
result_df['pIC50'].min()

0.4205594028602028

In [71]:
pt=[]


for i in result_df.pIC50:
  if float(i) >= -1:
      pt.append("Rank I")
  elif float(i) >= -4:
      pt.append("Rank II")
  else:
      pt.append("Rank III")

In [72]:
count_r1=0
for ele in pt:
  if ele=="Rank III":count_r1+=1

print(count_r1)

189


In [73]:
rank = pd.Series(pt, name='rank')
final_df = pd.concat([result_df, rank], axis=1)
final_df.head()

,Smiles,pIC50,rank
0,CC(C)C[C@@H]1NC(=O)C(C)(C)NC(=O)[C@H](CCCCCSSc...,6.638272,Rank I
1,O=C(Nc1cc(-c2ccccc2)ccc1O)c1ccccc1,4.301030,Rank I
2,C[C@H]1CN(c2ccc(C(=O)Nc3cc(-c4ccccc4)ccc3N)cn2...,4.301030,Rank I
3,Cc1cc(C)c(Nc2ccc(C(=O)N(C)CCCCCCC(=O)NO)cc2)c(...,7.455932,Rank I
4,COC(c1ccc(F)cc1)(c1ccc(F)cc1)c1ccc(C(=O)NCCCCC...,7.886057,Rank I


In [9]:
from rdkit import Chem
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import Descriptors
import pandas as pd

In [14]:
import pandas as pd

# Load your dataset (assuming it has a 'Smiles' column)
result_df = pd.read_csv('drugs_chembl.csv', sep=';', on_bad_lines='skip')  # Adjust file path and parameters as needed

# Check the contents to ensure it has the 'Smiles' column
print(result_df.head())  # Display the first few rows
print(result_df.columns)  # Check the column names

# Ensure there's a 'Smiles' column before proceeding
if 'Smiles' in result_df.columns:
    # Continue with descriptor calculation
    from rdkit import Chem
    from rdkit.ML.Descriptors import MoleculeDescriptors
    from rdkit.Chem import Descriptors

    descriptor_names = [desc_name[0] for desc_name in Descriptors._descList]
    calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)

    def calculate_descriptors(smiles):
        try:
            mol = Chem.MolFromSmiles(smiles)  # Convert SMILES to molecular object
            if mol is not None:
                return calculator.CalcDescriptors(mol)
            else:
                return [None] * len(descriptor_names)  # Return None for each descriptor if the molecule is None
        except:
            return [None] * len(descriptor_names)

    # Apply the function to the 'Smiles' column
    descriptors = result_df['Smiles'].apply(calculate_descriptors)
    desc = pd.DataFrame(descriptors.tolist(), columns=descriptor_names)  # Create DataFrame from descriptors
    results_df = pd.concat([result_df, desc], axis=1)  # Concatenate original DataFrame with descriptors DataFrame

    # Display the resulting DataFrame
    print(results_df)
else:
    print("The 'Smiles' column is not present in the DataFrame.")

  Molecule ChEMBL ID Molecule Name  Molecule Max Phase  Molecular Weight  \
0       CHEMBL253309           NaN                 NaN            563.79   
1       CHEMBL269935           NaN                 NaN            289.33   
2       CHEMBL437029           NaN                 NaN            521.62   
3      CHEMBL3639926           NaN                 NaN            411.55   
4      CHEMBL3693779           NaN                 NaN            496.55   

   #RO5 Violations  AlogP                             Compound Key  \
0              1.0   3.30  3, cyclo(-L-Am7(S2Py)-Aib-L-Leu-D-Pro-)   
1              0.0   4.31                                        2   
2              2.0   5.43                                       56   
3              0.0   4.88                               BDBM110013   
4              1.0   5.09                               BDBM110029   

                                              Smiles Standard Type  \
0  CC(C)C[C@@H]1NC(=O)C(C)(C)NC(=O)[C@H](CCCCCSSc...

Streaming output truncated to the last 5000 lines.
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:05:42] DEPRECATION WARNING: please use MorganGenerator
[06:0

     Molecule ChEMBL ID Molecule Name  Molecule Max Phase  Molecular Weight  \
0          CHEMBL253309           NaN                 NaN            563.79   
1          CHEMBL269935           NaN                 NaN            289.33   
2          CHEMBL437029           NaN                 NaN            521.62   
3         CHEMBL3639926           NaN                 NaN            411.55   
4         CHEMBL3693779           NaN                 NaN            496.55   
...                 ...           ...                 ...               ...   
6483           CHEMBL98    VORINOSTAT                 4.0            264.32   
6484      CHEMBL2364628  RICOLINOSTAT                 2.0            433.51   
6485      CHEMBL5186622           NaN                 NaN            407.87   
6486      CHEMBL5182624           NaN                 NaN            600.72   
6487      CHEMBL5206573           NaN                 NaN            541.63   

      #RO5 Violations  AlogP                       

In [15]:
cleaned_final = results_df.dropna()
print(f"Original DataFrame shape: {results_df.shape}")
print(f"Cleaned DataFrame shape: {cleaned_final.shape}")

Original DataFrame shape: (6488, 257)
Cleaned DataFrame shape: (0, 257)


In [18]:
import numpy as np
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator, X, y, title="Learning Curve", cv=5, n_jobs=None, train_sizes=np.linspace(0.1, 1.0, 5)):
    plt.figure(figsize=(10, 6))
    plt.title(title)
    plt.xlabel("Training Examples")
    plt.ylabel("Score")

    # Calculate learning curve
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring='accuracy')

    # Mean and standard deviation for training and validation scores
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    # Plot the learning curve
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")

    plt.legend(loc="best")
    plt.show()

# Plot learning curve for the RandomForestClassifier
plot_learning_curve(rf_classifier, X_train, y_train)

IndentationError: unexpected indent (<ipython-input-18-909490ea0880>, line 11)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
lazy_class = LazyClassifier()
train,test = lazy_class.fit(X_train, X_test, y_train, y_test)

# Print the results
print(train)